In [ ]:
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sqlalchemy import create_engine
from sqlalchemy import text
import json

# Download stopwords jika belum tersedia
nltk.download('stopwords')

# Koneksi ke MySQL via SQLAlchemy
engine = create_engine("mysql+mysqlconnector://root:@localhost/ikn-app")

# Load dataset CSV
df = pd.read_csv('dataset/ikn.csv')

# Simpan ke tabel 'dataset'
df.to_sql(name='dataset', con=engine, if_exists='replace', index=False)

# Filter hanya data dengan kata kunci
keywords = ['ikn', 'nusantara', 'ibu kota', 'ibukota', 'pemindahan', 'perpindahan']
filtered_df = df[df['full_text'].str.contains('|'.join(keywords), case=False, na=False)].copy()
filtered_df = filtered_df[filtered_df['full_text'].str.strip() != '']
filtered_df = filtered_df.drop_duplicates(subset='full_text')

# Inisialisasi Stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Load kamus kata dasar
kamus_path = 'sastrawi/kata-dasar.txt'
with open(kamus_path, 'r', encoding='utf-8') as f:
    kamus_sastrawi = set(word.strip() for word in f.readlines())

# Kata penting
kata_penting = {'ikn', 'nusantara', 'ibu', 'kota', 'ibukota', 'pemindahan', 'perpindahan'}

# Stopwords
stop_words = set(stopwords.words('indonesian'))
tambahan_stop = {
    'gw', 'gue', 'gua', 'lu', 'loe', 'lo', 'elu', 'nya', 'ya', 'aja', 'sih', 'lah', 'deh', 'dong',
    'kok', 'nih', 'tuh', 'lagi', 'kayak', 'gak', 'ga', 'nggak', 'ngga', 'yg', 'yang', 'saya', 'kamu'
}
stop_words.update(tambahan_stop)

# Kata sentimen
kata_positif = {'baik', 'bagus', 'maju', 'dukung', 'setuju', 'positif', 'indah', 'hebat'}
kata_negatif = {'tidak', 'buruk', 'tolak', 'negatif', 'korup', 'jelek', 'hancur', 'bencana', 'rusak'}

# Fungsi preprocessing
def preprocess_steps(text):
    # Bersihkan teks (hapus url, hashtag, mention, angka)
    data_clean = re.sub(r'http\S+|#\w+|@\w+|\d+', '', text)
    lower = data_clean.lower()
    no_punct = lower.translate(str.maketrans('', '', string.punctuation))
    replaced = re.sub(r'\bgw\b|\bgue\b|\bgua\b', 'saya', no_punct)
    replaced = re.sub(r'\blu\b|\bloe\b|\belo\b|\belu\b', 'kamu', replaced)
    replaced = re.sub(r'\bnggak\b|\bngga\b|\bga\b|\bgak\b', 'tidak', replaced)

    tokens = replaced.split()

    # Hapus stopword
    tokens_stop_removed = [w for w in tokens if w not in stop_words]

    # Gabungkan untuk stemming
    joined_for_stem = ' '.join(tokens_stop_removed)
    stemmed_text = stemmer.stem(joined_for_stem)

    # Token hasil stemming
    stemmed_tokens = stemmed_text.split()

    # Filtering: hanya kata di kamus atau penting
    tokens_filtered = [w for w in stemmed_tokens if w in kamus_sastrawi or w in kata_penting]

    # Urutkan
    tokens_sorted = sorted(tokens_filtered)

    # Gabungkan hasil akhir
    final_cleaned = ' '.join(tokens_sorted)

    # Deteksi sentimen
    sentimen = 'positif' if any(k in tokens_sorted for k in kata_positif) else (
        'negatif' if any(k in tokens_sorted for k in kata_negatif) else 'negatif'
    )

    return pd.Series([
        final_cleaned,
        lower,
        no_punct,
        json.dumps(tokens, ensure_ascii=False),
        json.dumps(tokens_stop_removed, ensure_ascii=False),
        json.dumps(tokens_sorted, ensure_ascii=False),
        sentimen
    ])

# Terapkan preprocessing
processed = filtered_df['full_text'].apply(preprocess_steps)
processed.columns = [
    'data_clean',
    'lowercasing',
    'remove_punctuation',
    'tokenizing',
    'stopword',
    'stemming',
    'sentimen'
]

# Filter hasil akhir: hanya yang terdiri dari >= 3 kata
processed['jumlah_kata'] = processed['data_clean'].str.split().str.len()
processed = processed[processed['jumlah_kata'] >= 3].drop(columns=['jumlah_kata'])

# Tambahkan kolom id sebagai primary key
processed.reset_index(drop=True, inplace=True)
processed.insert(0, 'id', processed.index + 1)

# Simpan ke MySQL
processed.to_sql(name='preprocessing', con=engine, if_exists='replace', index=False)

# Jadikan id sebagai PRIMARY KEY dan AUTO_INCREMENT
with engine.connect() as conn:
    conn.execute(text("""
        ALTER TABLE preprocessing
        MODIFY COLUMN id INT NOT NULL AUTO_INCREMENT,
        ADD PRIMARY KEY (id);
    """))

# Tampilkan hasil
print("✅ Data berhasil disimpan ke tabel MySQL: 'dataset', 'preprocessing'")
print("📌 Contoh data hasil pembersihan dan sentimen:")
print(processed[['id', 'stemming', 'sentimen']].head())
